# 감성분류를 위한 코드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
# 파이토치 라이브러리로, 딥러닝 모델을 만들고 학습하는 데 사용
import torch
# Hugging face dadasets 라이브러리로, 다양한 데이터셋을 쉽게 로드
from datasets import load_dataset
# Hugging face transformers 라이브러리로, 다양한 트랜스포머 모델을 로드 및 사용
from transformers import(
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
import pandas as pd
import numpy as np
# 모델 평가를 위한 라이브러리
import evaluate

In [ ]:
# 데이터 셋 로드
dataset = load_dataset('sepidmnorozy/Korean_sentiment')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/36000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1333 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2667 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 36000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1333
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 2667
    })
})

In [ ]:
# 데이터셋 출력
print(dataset['train'][3118])
print(dataset['train'][14310])

{'label': 1, 'text': '졸잼!!!성아가나중에억울한일이잇어서좀슬펏는데마지막은기쁘게끝나서다행이에여'}
{'label': 0, 'text': '진짜 어떻게 된놈의 영화가 여고괴담 1보다도 못함? 신기하다 그것도 2012년작이 1998년보다 못함 솔까 여고괴담1은 반전은 최고지 뭐 이놈의 영화는 여고괴담 시리즈보다도 못하는거같다'}


# 토큰화 Tokenize
한국어 감성 분석 모델을 훈련시키기 위한 데이터 전처리 및 준비 과정을 수행

In [ ]:
# 사용할 모델의 이름을 지정
model_name = 'kykim/bert-kor-base'

In [ ]:
# 한국어 텍스트 처리를 위한 BERT 모델
# 지정한 모델에 맞는 토크나이저를 로드
# 토크나이저는 텍스트 데이터를 BERT 모델이 처리할 수 있는 형식으로 변환하는 도구
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer
# 토크나이저의 설정 및 정보를 출력

# >> 위 코드를 실행하면 지정된 모델에 해당하는 사전 학습된 토크나이저를 로드

# AutoTokenizer
## Hugging Face의 Transformers 라이브러리에서 제공하는 클래스
## 제공된 모델 이름에 맞게 토크나이저 클래스를 자동으로 선택하고 로드
## 사전 학습된 토크나이저 로드
## 여러 모델에 대해 동일한 인터페이스를 제공하므로, 다양한 모델을 사용할 때 코드의 일관성 유지 가능

# from_pretrained
## 경로에 맞는 사전 학습된 토크나이저를 로드

tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='kykim/bert-kor-base', vocab_size=42000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
# 토크나이저 함수 정의
def tokenizer_func(x):
    return tokenizer(
        x['text'],
        padding='max_length', # 모든 텍스트를 최재 길이에 맞추어 패딩
        max_length=256,       # 최대 길이를 256으로 설정, 모델이 처리할 수 있는 입력의 최대 길이
        truncation=True       # 텍스트가 최대 길이를 초과할 경우 잘라내는 옵션
    )

# 로드된 토크나이저를 사용하여 입력 텍스트를 BERT 모델이 처리할 수 있는 형식으로 변환

In [ ]:
# 데이터셋의 각 샘플에 tokenizer_func를 적용 >> 토큰화된 데이터를 생성
tokenized_datasets = dataset.map(tokenizer_func, batched=True)

# map
## datasets 라이브러리에서 제공하는 강력한 기능으로,
## 데이터셋의 각 샘플에 대해 지정된 함수를 적용

# batched=True
## map 함수가 데이터를 샘플 단위가 아닌 배치 단위로 처리하도록 함
## 여러 샘플을 한 번에 처리하기 때문에 속도가 크게 향상

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1333 [00:00<?, ? examples/s]

Map:   0%|          | 0/2667 [00:00<?, ? examples/s]

In [ ]:
# 훈련데이터셋 샘플 수
train_num_samples = 10000

# train_num_samples 만큼의 샘플을 선택
# 모델 훈련 속도를 높이기 위해 훈련 데이터의 크기를 줄이는 데 사용
train_ds = tokenized_datasets['train'].shuffle(seed=42).select(range(train_num_samples))
eval_ds = tokenized_datasets['validation'].shuffle(seed=42)

# 전이학습 Transfer Learning
사전 학습된 모델을 새로운 작업에 재사용하여 학습 시간을 줄이고 성능을 향상시킴



---
### 사전학습(pre-training)

: 모델을 대규모 데이터셋에서 일반적인 특성을 학습하도록 훈련

### 미세조정(fine-tuning)

: 사전학습된 모델을 새로운 특정 작업에 맞게 조정


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
# AutoModelForSequenceClassification 클래스는 특정 시퀸스 분류 작업을 수행하도록 설계된 모델을 자동 로드
# from_pretrained : 지정된 모델 이름을 기반으로 사전 학습된 가중치를 모델에 로드
# num_labels=2 : 예측할 클래스(레이블)의 수를 지정

# 전이학습
## 허깅페이스 모델 허브에 사전 학습된 모델 가중치를 다운로드하고 로드
## 모델이 특정 작업에 맞게 설정(시퀀스 분류)

pytorch_model.bin:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kykim/bert-kor-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# HyperParams

In [ ]:
bs = 32       # 한 번의 훈련 단계에서 사용하는 데이터 샘플의 수
## 배치 크기가 클수록 한 번의 훈련단계에서 더 많은 샘플을 사용하지만, 메모리 사용량 증가
epochs = 4    # 전체 데이터셋을 몇 번 반복하여 훈련할지
lr = 1e-5     # 학습률로 모델이 가중치를 업데이트 하는 속도 조절
## 학습률이 너무 높으면 모델이 최적화되지 않을 수 있고, 너무 낮으면 학습이 너무 오래 걸릴 수 있음

In [ ]:
# 훈련 인자
args = TrainingArguments(
    'outputs',
    learning_rate = lr,
    warmup_ratio = 0.1,
    lr_scheduler_type = 'cosine',
    fp16 = True,
    evaluation_strategy='epoch',
    per_device_train_batch_size = bs,
    per_device_eval_batch_size = bs,
    gradient_accumulation_steps=4, #until bs =128
    eval_accumulation_steps = 4,
    num_train_epochs=epochs,
    weight_decay=0.01,
    report_to = 'none'
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Metrics

In [ ]:
# 성능 평가 지표
metric = evaluate.load('accuracy')

# all Transformers models retrun logits
def compute_metrics(eval_pred):
    logits, labels =eval_pred
    preds = np.argmax(logits, axis=-1)
    return metric.compute(predictions=preds, references=labels)

# 모델 예측값과 실제 라벨을 받아 정확도를 계산하는 함수

In [ ]:
class CustomTrainer(Trainer):
    def _save_checkpoint(self, model, trial, metrics=None):
        # 모델을 저장하기 전에 모든 텐서를 contiguous로 만듦
        for name, param in model.named_parameters():
            if param is not None:
                param.data = param.data.contiguous()
                if param.grad is not None:
                    param.grad.data = param.grad.data.contiguous()
        super()._save_checkpoint(model, trial, metrics)

# Trainer

In [ ]:
# 커스텀 트레이너 설정
# 모델, 훈련인자, 훈련 데이터셋, 평가 데이터셋, 토크나이저, 성능평가함수
trainer = CustomTrainer(
    model,
    args,
    train_dataset=train_ds, # 훈련데이터셋
    eval_dataset=eval_ds,   # 평가데이터셋
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

# Validation Loss : 모델이 학습하지 않은 데이터에 대해 얼마나 잘 예측하는지를 측정
##  loss 값이 감소하면 모델이 점점 더 나은 성능을 보여줌
##  증가 또는 일정하게 유지하면, 모델이 과적합되고 있을 가능성
##  >> 하이퍼파라미터 조정이나 데이터 전처리, 모델 구조의 변경 등을 고려

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.339661,0.867217
1,No log,0.310524,0.871718
2,No log,0.293890,0.878470
3,No log,0.292806,0.881470


TrainOutput(global_step=312, training_loss=0.35385823861146587, metrics={'train_runtime': 471.9333, 'train_samples_per_second': 84.758, 'train_steps_per_second': 0.661, 'total_flos': 5247486888099840.0, 'train_loss': 0.35385823861146587, 'epoch': 3.987220447284345})

In [ ]:
trainer.save_model('/content/drive/MyDrive/2조self/mymodels')

# 추론 Inference

In [ ]:
pipe = pipeline('text-classification', model='/content/drive/MyDrive/2조self/mymodels')
# 파이프라인 주요 구성요소
## 모델 : 사전 훈련된 모델이나 fine tuned된 모델
## 토크나이저 : 입력 텍스트를 모델이 이해할 수 있는 형태로 변환한 도구
## 모델 : 저장된 모델의 경로를 지정, 이 경로에서 모델을 로드하여 텍스트 분류 작업 수행

# pipline은 Hugging Face Transformers 라이브러리에서 제공하는 API, 특정 작업을 쉽게 수행

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


# 테스트셋 사용

In [ ]:
test_data = dataset['validation'].shuffle(seed=424)[:100]
td = pd.DataFrame(test_data)
td

,label,text
0,1,"비밀은 적정선에 관한 이야기이기도 하다. 사람에게 돈이란, 평판이란, 책임이란, 기..."
1,0,메인 시나리오 뼈대가 없고 보여주기라기엔 용과의 전투씬도 너무 부족함..더욱이 결말...
2,1,올??? 찌질한 스커드가 노만 리더스 행님이었다니 !! ㅋㅋ
3,0,5점이 적당하나 평점이 쓸데없이 높아서 낮출 필요가 있슴...
4,0,"연출, 기획, 스토리 전부다 쓰레기..제인생 영화중에 제일 쓰레기.. 절대보지마요진심"
...,...,...
95,1,"""""""철아 엄마가 꼭 니 신세 갚고 죽을께"""""""
96,0,이건 뭐 .. 완전 삼류네
97,0,으... 밑도 끝도 없음. 여자가 처음부터 자기혈청을 제공했으면 카오스가 안왔을거 아님?
98,0,요즘에 보기에는 상당히 지루한 진부한 스릴러. 원초적 본능을 뛰어 넘기에도 힘든 영...


In [ ]:
preds = pipe(td['text'].tolist())

preds_df = pd.DataFrame(preds)
preds_df

,label,score
0,LABEL_1,0.975715
1,LABEL_0,0.984877
2,LABEL_0,0.915608
3,LABEL_1,0.644758
4,LABEL_0,0.987627
...,...,...
95,LABEL_1,0.772595
96,LABEL_0,0.988571
97,LABEL_0,0.953846
98,LABEL_0,0.989112


In [ ]:
preds_df.rename(columns={'label':'pred'}, inplace=True)
preds_df['pred'] = preds_df['pred'].map({'LABEL_1': 1, 'LABEL_0':0})

preds_df = pd.concat([preds_df, td], axis=1)

In [ ]:
preds_df

,pred,score,label,text
0,1,0.975715,1,"비밀은 적정선에 관한 이야기이기도 하다. 사람에게 돈이란, 평판이란, 책임이란, 기..."
1,0,0.984877,0,메인 시나리오 뼈대가 없고 보여주기라기엔 용과의 전투씬도 너무 부족함..더욱이 결말...
2,0,0.915608,1,올??? 찌질한 스커드가 노만 리더스 행님이었다니 !! ㅋㅋ
3,1,0.644758,0,5점이 적당하나 평점이 쓸데없이 높아서 낮출 필요가 있슴...
4,0,0.987627,0,"연출, 기획, 스토리 전부다 쓰레기..제인생 영화중에 제일 쓰레기.. 절대보지마요진심"
...,...,...,...,...
95,1,0.772595,1,"""""""철아 엄마가 꼭 니 신세 갚고 죽을께"""""""
96,0,0.988571,0,이건 뭐 .. 완전 삼류네
97,0,0.953846,0,으... 밑도 끝도 없음. 여자가 처음부터 자기혈청을 제공했으면 카오스가 안왔을거 아님?
98,0,0.989112,0,요즘에 보기에는 상당히 지루한 진부한 스릴러. 원초적 본능을 뛰어 넘기에도 힘든 영...


In [ ]:
mask = preds_df['pred'] == preds_df['label']
len(preds_df[mask])

89

In [ ]:
import pandas as pd
path = '/content/drive/MyDrive/2조self/used_csv/expanded_skin_data_exploded.csv'
raw = pd.read_csv(path)
df = raw.copy()

In [ ]:
df.rename(columns={'indivi_review':'review_list'}, inplace=True)

In [ ]:
df_review = df['review_list']

In [ ]:
txts_td = pd.DataFrame(df_review)

In [ ]:
txts_td.head()

,review_list
0,보습력이 좋지만 지성피부가 쓰기엔 살짝 무거운감이잇다고 느껴졌어요 ㅠㅠ
1,유튜버분이 추천해주셔서 쓰게됐는데 아직 드라마틱한 변화는없지만 향이 좋음 한달뒤에 ...
2,모공에 좋다고 추천받아서 구매했어요
3,모공이 원래 좀 넓은 편이고 또 여름이라 더 걱정되기도 해서 열심히 사용해보려구요
4,순하기도 하고 향도 너무 좋아서 잘 사용 중이에요


In [ ]:
txts_td.to_csv('datas\데이터타입검사용(temp).csv')

In [ ]:
txts_td.dropna(inplace=True)

In [ ]:
txts_td.isnull().sum()

review_list    0
dtype: int64

In [ ]:
txts_td['review_list'] = txts_td['review_list'].astype(str)

In [ ]:
txts_td['review_list']

0                  보습력이 좋지만 지성피부가 쓰기엔 살짝 무거운감이잇다고 느껴졌어요 ㅠㅠ
1        유튜버분이 추천해주셔서 쓰게됐는데 아직 드라마틱한 변화는없지만 향이 좋음 한달뒤에 ...
2                                      모공에 좋다고 추천받아서 구매했어요
3            모공이 원래 좀 넓은 편이고 또 여름이라 더 걱정되기도 해서 열심히 사용해보려구요
4                              순하기도 하고 향도 너무 좋아서 잘 사용 중이에요
                               ...                        
31021    성분이 순해서 피부 자극도 없고 진정 효과도 있는 것 같아용기초 라인 바꾸고 난 이...
31022      저 후시다인 이제품사용해보고 반해가지고요즘 매일매일 사용하다 보니깐금방 떨어지더라고요
31023    보습력도 좋고순하디 순하면서 발림성도 좋은 그런 제품이라서너무 만족하면서 사용하고 ...
31024                              물토너인데도 불구하고 발림성이 뻐덕뻐덕해요
31025          자기 전 스킨케어로 썼더니 다음 날 얼굴 전체에 요철이 올라와서 바로 버렸어요
Name: review_list, Length: 31026, dtype: object

In [ ]:
# 정규화
import re
txts_td['review_list'].apply(lambda x : re.sub(r'[^ㄱ-ㅣ가-힣]+', ' ', x))

0                  보습력이 좋지만 지성피부가 쓰기엔 살짝 무거운감이잇다고 느껴졌어요 ㅠㅠ
1        유튜버분이 추천해주셔서 쓰게됐는데 아직 드라마틱한 변화는없지만 향이 좋음 한달뒤에 ...
2                                      모공에 좋다고 추천받아서 구매했어요
3            모공이 원래 좀 넓은 편이고 또 여름이라 더 걱정되기도 해서 열심히 사용해보려구요
4                              순하기도 하고 향도 너무 좋아서 잘 사용 중이에요
                               ...                        
31021    성분이 순해서 피부 자극도 없고 진정 효과도 있는 것 같아용기초 라인 바꾸고 난 이...
31022      저 후시다인 이제품사용해보고 반해가지고요즘 매일매일 사용하다 보니깐금방 떨어지더라고요
31023    보습력도 좋고순하디 순하면서 발림성도 좋은 그런 제품이라서너무 만족하면서 사용하고 ...
31024                              물토너인데도 불구하고 발림성이 뻐덕뻐덕해요
31025          자기 전 스킨케어로 썼더니 다음 날 얼굴 전체에 요철이 올라와서 바로 버렸어요
Name: review_list, Length: 31026, dtype: object

In [ ]:
txts_td['review_list']

0                  보습력이 좋지만 지성피부가 쓰기엔 살짝 무거운감이잇다고 느껴졌어요 ㅠㅠ
1        유튜버분이 추천해주셔서 쓰게됐는데 아직 드라마틱한 변화는없지만 향이 좋음 한달뒤에 ...
2                                      모공에 좋다고 추천받아서 구매했어요
3            모공이 원래 좀 넓은 편이고 또 여름이라 더 걱정되기도 해서 열심히 사용해보려구요
4                              순하기도 하고 향도 너무 좋아서 잘 사용 중이에요
                               ...                        
31021    성분이 순해서 피부 자극도 없고 진정 효과도 있는 것 같아용기초 라인 바꾸고 난 이...
31022      저 후시다인 이제품사용해보고 반해가지고요즘 매일매일 사용하다 보니깐금방 떨어지더라고요
31023    보습력도 좋고순하디 순하면서 발림성도 좋은 그런 제품이라서너무 만족하면서 사용하고 ...
31024                              물토너인데도 불구하고 발림성이 뻐덕뻐덕해요
31025          자기 전 스킨케어로 썼더니 다음 날 얼굴 전체에 요철이 올라와서 바로 버렸어요
Name: review_list, Length: 31026, dtype: object

In [ ]:
# 데이터프레임을 데이터셋으로 변환
from datasets import Dataset

dataset = Dataset.from_pandas(txts_td)
# Dataset 객체는 모델 훈련 및 평가 과정에서 사용하기 적합한 형식으로,
# 데이터 셋의 샘플을 빠르고 효율적으로 처리할 수 있도록 지원

In [ ]:
dataset

Dataset({
    features: ['review_list'],
    num_rows: 31026
})

In [ ]:
# 모델 이름과 토크나이저 초기화
model_name = 'kykim/bert-kor-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# 토크나이저 함수 정의
def tokenizer_func(examples):
    return tokenizer(
        examples['review_list'],
        padding='max_length',
        max_length=256,
        truncation=True
    )

In [ ]:
tokenized_review_list = dataset.map(tokenizer_func, batched=True)
# 데이터셋의 각 항목에 tokenizer_func을 적용하여 토크나이즈를 함
# batched=True는 배치 단위로 처리하여 속도를 높임

# tokenized_review_list에는 토크나이저를 사용하여 각 텍스트 샘플을 토큰화한 결과

Map:   0%|          | 0/31026 [00:00<?, ? examples/s]

In [ ]:
# 데이터셋 크기 확인
dataset_size = len(tokenized_review_list)
# train_num_samples = min(10000, dataset_size)  # 데이터셋 크기를 넘지 않도록 설정

In [ ]:
txts_td = tokenized_review_list.select(range(dataset_size))

# tokenized_review_list에는 토크나이저를 사용하여 각 텍스트 샘플을 토큰화한 결과
# select 메서드는 특정 샘플을 선택하거나 필터링할 때 사용

In [ ]:
txts_td

Dataset({
    features: ['review_list', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 31026
})

In [ ]:
preds_txts = pipe(txts_td['review_list'])

In [ ]:
preds_txts_df = pd.DataFrame(preds_txts)

In [ ]:
preds_txts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31026 entries, 0 to 31025
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   31026 non-null  object 
 1   score   31026 non-null  float64
dtypes: float64(1), object(1)
memory usage: 484.9+ KB


In [ ]:
preds_txts_df.head()

,label,score
0,LABEL_1,0.766785
1,LABEL_1,0.905482
2,LABEL_1,0.757586
3,LABEL_1,0.942523
4,LABEL_1,0.956111


In [ ]:
preds_txts_df['label'] = preds_txts_df['label'].map({'LABEL_1':1, 'LABEL_0':0})

In [ ]:
review_labeling_df = pd.DataFrame(txts_td)

In [ ]:
review_labeling_df.head()

,review_list,input_ids,token_type_ids,attention_mask
0,보습력이 좋지만 지성피부가 쓰기엔 살짝 무거운감이잇다고 느껴졌어요 ㅠㅠ,"[2, 38465, 18067, 36604, 8048, 16325, 8075, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,유튜버분이 추천해주셔서 쓰게됐는데 아직 드라마틱한 변화는없지만 향이 좋음 한달뒤에 ...,"[2, 39808, 14324, 22137, 19485, 21190, 30744, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,모공에 좋다고 추천받아서 구매했어요,"[2, 16821, 8008, 15333, 14195, 23396, 16088, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,모공이 원래 좀 넓은 편이고 또 여름이라 더 걱정되기도 해서 열심히 사용해보려구요,"[2, 32297, 14627, 6096, 16321, 27927, 3547, 32...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,순하기도 하고 향도 너무 좋아서 잘 사용 중이에요,"[2, 5103, 14889, 14042, 16354, 14000, 15044, 5...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."


In [ ]:
# review_labeling_df.drop(columns=['__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'], inplace=True)

In [ ]:
preds_txts_df = pd.concat([preds_txts_df, review_labeling_df], axis=1)
preds_txts_df

,label,score,review_list,input_ids,token_type_ids,attention_mask
0,1,0.766785,보습력이 좋지만 지성피부가 쓰기엔 살짝 무거운감이잇다고 느껴졌어요 ㅠㅠ,"[2, 38465, 18067, 36604, 8048, 16325, 8075, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,1,0.905482,유튜버분이 추천해주셔서 쓰게됐는데 아직 드라마틱한 변화는없지만 향이 좋음 한달뒤에 ...,"[2, 39808, 14324, 22137, 19485, 21190, 30744, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,1,0.757586,모공에 좋다고 추천받아서 구매했어요,"[2, 16821, 8008, 15333, 14195, 23396, 16088, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,1,0.942523,모공이 원래 좀 넓은 편이고 또 여름이라 더 걱정되기도 해서 열심히 사용해보려구요,"[2, 32297, 14627, 6096, 16321, 27927, 3547, 32...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1,0.956111,순하기도 하고 향도 너무 좋아서 잘 사용 중이에요,"[2, 5103, 14889, 14042, 16354, 14000, 15044, 5...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
31021,1,0.984028,성분이 순해서 피부 자극도 없고 진정 효과도 있는 것 같아용기초 라인 바꾸고 난 이...,"[2, 16796, 31544, 14081, 33868, 14771, 16683, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
31022,1,0.792353,저 후시다인 이제품사용해보고 반해가지고요즘 매일매일 사용하다 보니깐금방 떨어지더라고요,"[2, 6015, 7876, 16428, 8159, 28403, 14874, 188...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
31023,1,0.984652,보습력도 좋고순하디 순하면서 발림성도 좋은 그런 제품이라서너무 만족하면서 사용하고 ...,"[2, 15617, 16111, 14410, 8233, 8052, 8264, 510...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
31024,0,0.716591,물토너인데도 불구하고 발림성이 뻐덕뻐덕해요,"[2, 4223, 26971, 18764, 16384, 30434, 4733, 88...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."


In [ ]:
preds_txts_df[preds_txts_df.label == 0]

,label,score,review_list,input_ids,token_type_ids,attention_mask
15,0,0.607663,저는 민감성홍조수분부족 타입이에요,"[2, 14156, 26786, 8540, 8181, 22052, 23947, 33...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
26,0,0.806692,콧물 제형은 아닙니다,"[2, 6971, 8050, 36986, 18002, 3, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
28,0,0.789876,알콜향이 딱히 나지도 않아요,"[2, 27692, 15417, 19151, 19293, 8144, 15724, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
73,0,0.689982,물론 무향을 선호하는지라 그 부분이 살짝 아쉽기는 하지만 토너 특성상 금방 날아갑니다,"[2, 14268, 29544, 8051, 21359, 18658, 2430, 15...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
76,0,0.857675,근데 뚜껑이 원래 이렇게 헐거운 건지 모르겠네요ㅜ,"[2, 14441, 24052, 14627, 14063, 7729, 14901, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
30943,0,0.694031,워낙 귀차니즘이 심해서 토너를 화장솜에 적셔서 사용하는 것도 번거롭더라구요ㅠㅠ,"[2, 15490, 35006, 8018, 26756, 17638, 8273, 37...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ..."
30960,0,0.651804,추워진 날씨에 피부부터 바로 반응이 오더라고요,"[2, 19678, 8250, 22311, 14081, 14008, 14067, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
30961,0,0.648448,갑자기 뒤집어져서 기초제품부터 바로 바꿨어요,"[2, 15612, 21922, 14489, 16144, 14309, 14008, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
31024,0,0.716591,물토너인데도 불구하고 발림성이 뻐덕뻐덕해요,"[2, 4223, 26971, 18764, 16384, 30434, 4733, 88...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."


In [ ]:
preds_txts_df.to_csv('/content/drive/MyDrive/2조self/skin_review_labeling.csv')

In [ ]:
path = '/content/drive/MyDrive/2조self/skin_review_labeling.csv'
df = pd.read_csv(path)

In [ ]:

# 같은 행에서 df['label']의 값이 1이면 df['review_list']에 [긍정]이라는 텍스트를 추가
df.loc[df['label'] == 1, 'review_list'] = '[긍정] ' + df['review_list']

# 같은 행에서 df['label']의 값이 0이면 df['review_list']에 [부정]이라는 텍스트를 추가
df.loc[df['label'] == 0, 'review_list'] = '[부정] ' + df['review_list']

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31026 entries, 0 to 31025
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      31026 non-null  int64  
 1   label           31026 non-null  int64  
 2   score           31026 non-null  float64
 3   review_list     31026 non-null  object 
 4   input_ids       31026 non-null  object 
 5   token_type_ids  31026 non-null  object 
 6   attention_mask  31026 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 1.7+ MB


In [ ]:
# df.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
df.reset_index(drop=True)

,Unnamed: 0,label,score,review_list,input_ids,token_type_ids,attention_mask
0,0,1,0.766785,[긍정] 보습력이 좋지만 지성피부가 쓰기엔 살짝 무거운감이잇다고 느껴졌어요 ㅠㅠ,"[2, 38465, 18067, 36604, 8048, 16325, 8075, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,1,1,0.905482,[긍정] 유튜버분이 추천해주셔서 쓰게됐는데 아직 드라마틱한 변화는없지만 향이 좋음 ...,"[2, 39808, 14324, 22137, 19485, 21190, 30744, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,2,1,0.757586,[긍정] 모공에 좋다고 추천받아서 구매했어요,"[2, 16821, 8008, 15333, 14195, 23396, 16088, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,1,0.942523,[긍정] 모공이 원래 좀 넓은 편이고 또 여름이라 더 걱정되기도 해서 열심히 사용해...,"[2, 32297, 14627, 6096, 16321, 27927, 3547, 32...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,4,1,0.956111,[긍정] 순하기도 하고 향도 너무 좋아서 잘 사용 중이에요,"[2, 5103, 14889, 14042, 16354, 14000, 15044, 5...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
31021,31021,1,0.984028,[긍정] 성분이 순해서 피부 자극도 없고 진정 효과도 있는 것 같아용기초 라인 바꾸...,"[2, 16796, 31544, 14081, 33868, 14771, 16683, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
31022,31022,1,0.792353,[긍정] 저 후시다인 이제품사용해보고 반해가지고요즘 매일매일 사용하다 보니깐금방 떨...,"[2, 6015, 7876, 16428, 8159, 28403, 14874, 188...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
31023,31023,1,0.984652,[긍정] 보습력도 좋고순하디 순하면서 발림성도 좋은 그런 제품이라서너무 만족하면서 ...,"[2, 15617, 16111, 14410, 8233, 8052, 8264, 510...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
31024,31024,0,0.716591,[부정] 물토너인데도 불구하고 발림성이 뻐덕뻐덕해요,"[2, 4223, 26971, 18764, 16384, 30434, 4733, 88...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."


In [ ]:
df.to_csv('/content/drive/MyDrive/2조self/skin_review_labeling.csv')